In [ ]:
import cv2

img = cv2.imread('200.jpeg')

In [ ]:
import IPython

def imshow(img):
    returnValue, buffer = cv2.imencode('.jpeg', img)
    IPython.display.display(IPython.display.Image(data=buffer.tobytes()))

imshow(img)

height, width, channels = img.shape
img.shape

In [3]:
import numpy as np
import time

def ps_filter(img):
    #Sobel Hor filter  
    kernelV    = np.array([[-1.0,-2.0,-1.0],[0.0,0.0,0.0],[1.0,2.0,1.0]],np.float32)
    kernelH    = np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)
    kernelG    = np.array([[1.0,2.0,1.0],[2.0,4.0,2.0],[1.0,2.0,1.0]],np.float32)
    kernelD   = np.ones((3,3),np.uint8)
    buf       = np.ones((height,width),np.uint8)
    frame_out = np.ones((height,width),np.uint8)

    num_frames    = 20

    gaussian_kernel = cv2.getGaussianKernel(5,3)
#     print(gaussian_kernel)
    # kernel_2D = gaussian_kernel @ gaussian_kernel.transpose()

    start = time.time()
    frame_in_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # cv2.filter2D(img, -1, kernelG, buf, borderType=cv2.BORDER_CONSTANT)
    blurred=cv2.filter2D(img, -1, gaussian_kernel)
#     cv2.dilate(buf, kernelD, frame_out, borderType=cv2.BORDER_CONSTANT)
#     imshow(blurred)

    time_sw_total = time.time() - start
    print("Time Taken: {} ".format(time_sw_total))

In [4]:
# print(open('prunSW','r').read())
# res = !cat prunSW | grep filter2D | awk '{{print $$2}}'
# tottime_sw_filter2d = float(res[0])
# res = !cat prunSW | grep dilate   | awk '{{print $$2}}'
# tottime_sw_dilate   = float(res[0])

In [5]:
import cv2 #NOTE: This needs to be loaded first

# Load filter2D + dilate overlay
from pynq import Overlay
bs = Overlay("/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.bit")
bs.download()
import pynq_cv.overlays.xv2Filter2DDilate as xv2

# Load xlnk memory manager
from pynq import Xlnk
Xlnk.set_allocator_library('/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.so')
mem_manager = Xlnk()

In [6]:
import numpy as np
import time
import multiprocessing
from multiprocessing.pool import ThreadPool

def stream_fpga_filter(img):
    kernelF    = np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)
    kernelVoid = np.zeros(0)
    xFin       = mem_manager.cma_array((height,width),np.uint8)
    xFbuf      = mem_manager.cma_array((height,width),np.uint8)
    xFout      = mem_manager.cma_array((height,width),np.uint8)

#     num_frames       = 20

#     xFin[:]    = frame_in_gray[:]
    start = time.time()
    for _ in range(10):
        frame_in_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        xFin[:]       = frame_in_gray[:]
        
        xv2.filter2D(xFin, -1, kernelF, xFbuf, borderType=cv2.BORDER_CONSTANT)
        xv2.dilate(xFbuf, kernelVoid, xFout, borderType=cv2.BORDER_CONSTANT)
        
#         frame_out[:]  = xFout[:]
#         imshow(xFout[:])
    time_hw_total = time.time() - start
    print("Time Taken:  " + str(time_hw_total))

def fpga_filter(img):
    kernelF    = np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)
    kernelVoid = np.zeros(0)
    xFin       = mem_manager.cma_array((height,width),np.uint8)
    xFbuf      = mem_manager.cma_array((height,width),np.uint8)
    xFout      = mem_manager.cma_array((height,width),np.uint8)

#     num_frames       = 20

#     xFin[:]    = frame_in_gray[:]
    start = time.time()
    frame_in_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    xFin[:]       = frame_in_gray[:]

    xv2.filter2D(xFin, -1, kernelF, xFbuf, borderType=cv2.BORDER_CONSTANT)
    xv2.dilate(xFbuf, kernelVoid, xFout, borderType=cv2.BORDER_CONSTANT)

#         frame_out[:]  = xFout[:]
#         imshow(xFout[:])
    time_hw_total = time.time() - start
    print("Time Taken:  " + str(time_hw_total))

# Single Processing

In [7]:
print('time taken for small image')
num_iterations=10
print("Time taken on PS with Single Processing for {} Iterations".format(num_iterations))
start_time=time.time()
for i in range(num_iterations):
    ps_filter(img)
print('Time taken for non-parallel execution: {}'.format(time.time()-start_time))

num_iterations=10
print("Time taken on FPGA with stream Single Processing for {} Iterations".format(num_iterations))
start_time=time.time()
# for i in range(num_iterations):
stream_fpga_filter(img)
print('Time taken for non-parallel execution: {}'.format(time.time()-start_time))

print("Time taken on FPGA with stream Single Processing for {} Iterations".format(num_iterations))
start_time=time.time()
for i in range(num_iterations):
    fpga_filter(img)
print('Time taken for non-parallel execution: {}'.format(time.time()-start_time))


time taken for small image
Time taken on PS with Single Processing for 10 Iterations
Time Taken: 0.005842924118041992 
Time Taken: 0.00412297248840332 
Time Taken: 0.004113674163818359 
Time Taken: 0.0041277408599853516 
Time Taken: 0.004109859466552734 
Time Taken: 0.004124164581298828 
Time Taken: 0.004167079925537109 
Time Taken: 0.004114389419555664 
Time Taken: 0.004142045974731445 
Time Taken: 0.004122257232666016 
Time taken for non-parallel execution: 0.05222630500793457
Time taken on FPGA with stream Single Processing for 10 Iterations
Time Taken:  0.008664846420288086
Time taken for non-parallel execution: 0.019890785217285156
Time taken on FPGA with stream Single Processing for 10 Iterations
Time Taken:  0.0009882450103759766
Time Taken:  0.0008890628814697266
Time Taken:  0.0008792877197265625
Time Taken:  0.0008680820465087891
Time Taken:  0.0008800029754638672
Time Taken:  0.0008678436279296875
Time Taken:  0.0008754730224609375
Time Taken:  0.0008988380432128906
Time Tak

# Multi-Processing

In [8]:
print("Time taken on PS with Multi-Processing for {} Iterations".format(num_iterations))
pool=multiprocessing.Pool()
start_time=time.time()
pool.map(ps_filter, [img]*num_iterations)
print('Time taken for multi-processing= {}'.format(time.time()-start_time))


print("Time taken on FPGA with Multi-Processing for {} Iterations".format(num_iterations))
pool=multiprocessing.Pool()
start_time=time.time()
pool.map(fpga_filter, [img]*num_iterations)
print('Time taken for multi-processing= {}'.format(time.time()-start_time))

Time taken on PS with Multi-Processing for 10 Iterations
Time Taken: 0.004774332046508789 
Time Taken: 0.0053637027740478516 
Time Taken: 0.0051898956298828125 
Time Taken: 0.005769252777099609 
Time Taken: 0.00951528549194336 
Time Taken: 0.0045626163482666016 
Time Taken: 0.004465579986572266 
Time Taken: 0.004378795623779297 
Time Taken: 0.0045261383056640625 
Time Taken: 0.007013559341430664 
Time taken for multi-processing= 0.06764793395996094
Time taken on FPGA with Multi-Processing for 10 Iterations
Time Taken:  0.003299713134765625
Time Taken:  0.003304719924926758
Time Taken:  0.0044002532958984375
Time Taken:  0.001390218734741211
Time Taken:  0.0009567737579345703
Time Taken:  0.001710653305053711
Time Taken:  0.0018153190612792969
Time Taken:  0.001127004623413086
Time Taken:  0.0010175704956054688
Time Taken:  0.0011069774627685547
Time taken for multi-processing= 0.06943035125732422
